In [1]:
from docx import Document

document = Document('./tax.docx') 
full_text = ''

for index, paragraph in enumerate(document.paragraphs):
    full_text += f'{paragraph.text}\n' # 1.문서 내용을 읽어옴.


In [ ]:
import tiktoken

def split_text(full_text, chunk_size):
    tokenizer = tiktoken.encoding_for_model("gpt-4o") # 해당 모델의 인코딩 방식을 사용하는 토크나이저를 생성함.
    total_token = tokenizer.encode(full_text) # 텍스트를 토큰으로 바꿈.
    total_token_count = len(total_token)
    text_list = []
    
    for i in range(0, total_token_count, chunk_size):
        chunk = total_token[i: i+chunk_size]
        text = tokenizer.decode(chunk) # 토큰을 텍스트로 바꿈.
        text_list.append(text)
        
    return text_list

In [4]:
chunk_list = split_text(full_text, 1500) # 2. 문서를 쪼갬.

In [5]:
import chromadb

chroma_client = chromadb.Client() # ChromaDB 클라이언트를 생성함. (Chroma엔진을 직접 제어할 수 있음.)

In [6]:
collection_name = "tax_collection"

tax_collection = chroma_client.create_collection(collection_name) # 문서를 저장하기 위한 컬렉션(테이블)을 생성함.

In [7]:
import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()

oepnai_api_key = os.getenv("OPENAI_API_KEY")

openai_embedding = OpenAIEmbeddingFunction(api_key=oepnai_api_key, 
                                           model_name='text-embedding-3-large')

In [ ]:
# 3.임베딩하여 데이터베이스에 저장함.

tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)
# 해당 컬렉션이 이미 존재할 경우, 이를 가져옴.(get)
# 해당 컬렉션이 존재하지 않을 경우, 새 컬렉션을 생성함.(create)

# 문서와 질문(쿼리)에 모두에 해당 임베딩을 적용함.

In [ ]:
id_list = []

for index in range(len(chunk_list)):
    id_list.append(f'{index}')

In [ ]:
tax_collection.add(documents=chunk_list, ids=id_list)
# 문서 조각을 벡터로 변환한 후, 컬렉션에 추가함.
# 고유 ID와 함께 저장함으로써 중복을 방지함.

In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

retrieved_docs = tax_collection.query(query_texts=query, n_results=3) # 4.질문에 대한 유사도를 검색함.
# 질문(쿼리)을 벡터로 변환한 후, 의미적으로 가장 가까운 문서 조각 3개를 컬렉션에서 찾아옴.

In [62]:
retrieved_docs['documents'][0]

['률」에 따른 소싸움경기투표권 및 「국민체육진흥법」에 따른 체육진흥투표권의 구매자가 받는 환급금(발생 원인이 되는 행위의 적법 또는 불법 여부는 고려하지 아니한다)\n5. 저작자 또는 실연자(實演者)ㆍ음반제작자ㆍ방송사업자 외의 자가 저작권 또는 저작인접권의 양도 또는 사용의 대가로 받는 금품\n6. 다음 각 목의 자산 또는 권리의 양도ㆍ대여 또는 사용의 대가로 받는 금품\n가. 영화필름\n나. 라디오ㆍ텔레비전방송용 테이프 또는 필름\n다. 그 밖에 가목 및 나목과 유사한 것으로서 대통령령으로 정하는 것\n7. 광업권ㆍ어업권ㆍ양식업권ㆍ산업재산권ㆍ산업정보, 산업상 비밀, 상표권ㆍ영업권(대통령령으로 정하는 점포 임차권을 포함한다), 토사석(土砂石)의 채취허가에 따른 권리, 지하수의 개발ㆍ이용권, 그 밖에 이와 유사한 자산이나 권리를 양도하거나 대여하고 그 대가로 받는 금품\n8. 물품(유가증권을 포함한다) 또는 장소를 일시적으로 대여하고 사용료로서 받는 금품\n8의2. 「전자상거래 등에서의 소비자보호에 관한 법률」에 따라 통신판매중개를 하는 자를 통하여 물품 또는 장소를 대여하고 대통령령으로 정하는 규모 이하의 사용료로서 받은 금품\n9. 「공익사업을 위한 토지 등의 취득 및 보상에 관한 법률」 제4조에 따른 공익사업과 관련하여 지역권ㆍ지상권(지하 또는 공중에 설정된 권리를 포함한다)을 설정하거나 대여함으로써 발생하는 소득\n10. 계약의 위약 또는 해약으로 인하여 받는 소득으로서 다음 각 목의 어느 하나에 해당하는 것\n가. 위약금\n나. 배상금\n다. 부당이득 반환 시 지급받는 이자\n11. 유실물의 습득 또는 매장물의 발견으로 인하여 보상금을 받거나 새로 소유권을 취득하는 경우 그 보상금 또는 자산\n12. 소유자가 없는 물건의 점유로 소유권을 취득하는 자산\n13. 거주자ㆍ비거주자 또는 법인의 대통령령으로 정하는 특수관계인이 그 특수관계로 인하여 그 거주자ㆍ비거주자 또는 법인으로부터 받는 경제적 이익으로서 급여ㆍ배당 또는 증여로 보지 아니하는 금품\n14.

In [ ]:
from openai import OpenAI

client = OpenAI() # OpenAI 클라이언트를 생성함.

# 5.유사도 검색 문서와 질문을 LLM에게 전달함.
answer = client.chat.completions.create(
    model="gpt-4o",
    messages=[ 
        {"role": "system" , "content": f"당신은 한국의 소득세 전문가입니다. 다음의 내용을 참고해서 사용자의 질문에 답변해주세요.{retrieved_docs['documents'][0]}"}, # AI가 수행할 역할을 지정함.
        {"role": "user" , "content": query} # 사용자의 질문(쿼리)을 작성함.
    ]
)

In [ ]:
answer.choices[0].message.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해 몇 가지 가정이 필요합니다. 기본적으로 소득세는 과세표준에 따라 누진세율이 적용됩니다. 한국의 소득세율은 다음과 같습니다(2023년 기준):\n\n- 1,200만원 이하: 6%\n- 1,200만원 초과 ~ 4,600만원 이하: 15%\n- 4,600만원 초과 ~ 8,800만원 이하: 24%\n- 8,800만원 초과 ~ 1억5천만원 이하: 35%\n- 1억5천만원 초과 ~ 3억원 이하: 38%\n- 3억원 초과 ~ 5억원 이하: 40%\n- 5억원 초과: 42%\n\n여기에 추가적으로 주민세(소득세의 10%)가 부과됩니다.\n\n과세표준을 계산하기 위해서는 총급여에서 각종 소득공제를 차감해야 합니다. 일반적으로 근로소득공제와 기본적인 인적공제를 고려할 수 있습니다만, 정확한 계산을 위해서는 개인의 세부적인 소득 및 공제 상황을 알아야 합니다. \n\n예를 들어, 기본적인 근로소득공제와 인적공제만 간단히 적용해본다면:\n1. 근로소득공제: 총급여가 5천만원인 경우 약 1,100만원 정도.\n2. 인적공제: 본인에 대한 기본 공제로 150만원.\n\n과세표준 = 총급여 - 근로소득공제 - 인적공제 = 5,000만원 - 1,100만원 - 150만원 = 3,750만원.\n\n이 과세표준에 대해 소득세를 계산해보면:\n- 1,200만원까지 6% 적용: 1,200만원 x 6% = 72만원\n- 1,200만원 초과 ~ 3,750만원까지 15% 적용: (3,750만원 - 1,200만원) x 15% = 382.5만원\n\n따라서, 소득세는 72만원 + 382.5만원 = 454.5만원이 됩니다.\n\n여기에 주민세(소득세의 10%)가 추가되면: 454.5만원 x 10% = 45.45만원.\n\n최종적으로 소득세와 주민세 합계는: 454.5만원 + 45.45만원 = 약 500만원입니다.\n\n그러나, 이는 기본적인 예시이며, 개인의 구체적인 공제 항목에 따라 달라질 수 있습니다. 더욱 정확한 계산을 위해서는 세무 전문가와 상담하시기